## Imports and Grab Data

In [1]:
from utils.notebook_2_utils import * 
import utils.notebook_2_utils as utils

training_df = pd.read_csv("data/training.csv").iloc[:100]
lookup_df = pd.read_csv("data/IdLookupTable.csv")
training_df.fillna(method = 'ffill',inplace = True)

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [2]:
train_X, train_Y, val_X, val_Y = create_train_test_sets_nchw(training_df, normalize=True)

## ResNet Model
* Input shape:  64, 1, 96, 96 (nchw)
* Output shape: 64, 30 (nc)


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
# from sklearn.model_selection import train_test_split
import torch.nn.functional as F

### Model Definition

In [4]:
resnet = models.resnet18(pretrained=True)


/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

## Control Panel

In [6]:
EPOCHS = 40
LEARNING_RATE = .02
BATCH_SIZE = 32


loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.SGD(resnet.parameters(), lr=LEARNING_RATE)





## Replace first and Last layers
* my input shape [32, 1, 96, 96] (NCHW) channels first

In [8]:
# freeze all layers
for param in resnet.parameters():
    param.requires_grad = False

# replace last layer
resnet.fc = nn.Linear(512, 30) # 30 is the number of keypoints we want to predict

# move to GPU
resnet = resnet.to(device)


### DataLoaders

In [9]:
torch.manual_seed(42)
from torch.utils.data import TensorDataset, DataLoader


# Put data to target device
train_X, train_Y = train_X.to(device), train_Y.to(device)
val_X, val_Y = val_X.to(device), val_Y.to(device)

train_dataset = TensorDataset(train_X, train_Y)
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

test_dataset = TensorDataset(val_X, val_Y)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [16]:
for X_batch, y_batch in train_dataloader:
    print(X_batch.shape, y_batch.shape)
    # break

torch.Size([32, 1, 96, 96]) torch.Size([32, 30])
torch.Size([32, 1, 96, 96]) torch.Size([32, 30])
torch.Size([16, 1, 96, 96]) torch.Size([16, 30])


In [17]:
preds = resnet(X_batch)

RuntimeError: Given groups=1, weight of size [64, 3, 7, 7], expected input[16, 1, 96, 96] to have 3 channels, but got 1 channels instead

In [ ]:
y_batch.shape

In [ ]:
loss_fn(preds, y_batch)

## Training Loop (No gradient accumilation, see below for loop that's used)

In [ ]:
for epoch in range(EPOCHS):
    ### Training mode 
    cnn.train()

    
    for X_batch, y_batch in train_dataloader:
        # 1. Forward pass (model outputs raw logits)
        y_logits = cnn(X_batch)
        
        # 2. Calculate loss/accuracy
        loss = loss_fn(y_logits, y_batch)
        
        # 3. Optimizer zero grad
        optimizer.zero_grad()
        
        # 4. Loss backwards
        loss.backward()
        
        # 5. Optimizer step
        optimizer.step()
    
    
    cnn.eval()
    with torch.inference_mode():
        for X_batch, y_batch in test_dataloader:
            test_logits = cnn(X_batch)
            test_loss = loss_fn(test_logits, y_batch)

    # Print out what's happening
    if epoch % 10 == 0:
        print(f"Epoch: {epoch} | Loss: {loss:.5f} | Test Loss: {test_loss:.5f}")

In [ ]:
preds = cnn(val_X)

In [ ]:
preds.shape

In [ ]:
val_Y.shape

In [ ]:
loss_fn(preds.to(device), val_Y.to(device))

In [ ]:
val_X.shape, preds.shape, val_Y.shape

In [ ]:
preds[0][1]

In [ ]:
val_X[1].shape, val_X[1][0].shape

In [ ]:
def show_pred(X, preds, actual, index, point):
    plt.imshow(X[index][0],cmap='gray')

    plt.scatter(96* preds[index][point],96* preds[index][point + 1] ,c='r', marker='s', s=60, alpha=.5)
    plt.scatter(96*actual[index][point],96* actual[index][point+1],c='g', marker='s', s=60, alpha=.5)

    plt.legend(['predicted','actual'])
    


In [ ]:
show_pred(val_X.to("cpu"), preds.to("cpu").detach().numpy(), val_Y.to("cpu").detach().numpy(), 0, 0)

In [ ]:
preds[0][0], preds[0][1]

In [ ]:
val_Y[0][0], val_Y[0][1]

In [ ]:
val_X[0][0]